In [1]:
import requests
import pandas as pd
import pypyodbc as odbc
import time

# --- 1️⃣ Connect to SQL Server ---
connection_string = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={r'Akeelah\SQLEXPRESS'};"
    f"DATABASE=Crypto_DataWarehouse;"
    "Trusted_Connection=yes;"
)

conn = odbc.connect(connection_string)

# --- 2️⃣ Fetch all coins from the DB ---
coins_df = pd.read_sql(
    "SELECT DISTINCT id, name FROM bronze.coin_market WHERE id IS NOT NULL", 
    conn
)

print(f"Found {len(coins_df)} coins in the database.")

# --- 3️⃣ Define function to fetch CoinGecko data ---
def connecting_coin_market_api(coin_id):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}"
    params = {
        "localization": False,  # exclude all languages except 'en'
        "sparkline": False,
        "market_data": False    # skip market data
    }
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        print(f"Fetched {coin_id} successfully")
        return data
    except Exception as e:
        print(f"Error fetching {coin_id}: {e}")
        return None

# --- 4️⃣ Loop through coins and fetch data ---
all_data = []

for index, row in coins_df.iterrows():
    coin_id = row['id']
    result = connecting_coin_market_api(coin_id)
    if result:
        all_data.append(result)
    time.sleep(15)  # polite delay to avoid rate limits

print(f"Fetched data for {len(all_data)} coins from CoinGecko.")

# --- 5️⃣ Convert to DataFrame for parent coin table ---
coin_df = pd.json_normalize(all_data)
coin_df

C:\Users\eBay\AppData\Local\Temp\ipykernel_5852\3550357962.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  coins_df = pd.read_sql(


Found 50 coins in the database.
Fetched aave successfully
Fetched avalanche-2 successfully
Fetched binance-bridged-usdt-bnb-smart-chain successfully
Fetched binancecoin successfully
Fetched bitcoin successfully
Fetched bitcoin-cash successfully
Fetched bittensor successfully
Fetched cardano successfully
Fetched chainlink successfully
Fetched coinbase-wrapped-btc successfully
Fetched crypto-com-chain successfully
Fetched dai successfully
Fetched dogecoin successfully
Fetched ethena-staked-usde successfully
Fetched ethena-usde successfully
Fetched ethereum successfully
Fetched figure-heloc successfully
Fetched hedera-hashgraph successfully
Fetched hyperliquid successfully
Fetched leo-token successfully
Fetched litecoin successfully
Fetched mantle successfully
Fetched memecore successfully
Fetched monero successfully
Fetched paypal-usd successfully
Fetched polkadot successfully
Fetched rain successfully
Fetched ripple successfully
Fetched shiba-inu successfully
Fetched solana successfully

,id,symbol,name,web_slug,asset_platform_id,block_time_in_minutes,hashing_algorithm,categories,preview_listing,public_notice,...,detail_platforms.terra-2.decimal_place,detail_platforms.terra-2.contract_address,platforms.telos,detail_platforms.telos.decimal_place,detail_platforms.telos.contract_address,detail_platforms.telos.geckoterminal_url,platforms.initia,detail_platforms.initia.decimal_place,detail_platforms.initia.contract_address,detail_platforms.initia.geckoterminal_url
0,aave,aave,Aave,aave,ethereum,0,None,"[Decentralized Finance (DeFi), Yield Farming, ...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,avalanche-2,avax,Avalanche,avalanche,None,0,None,"[Smart Contract Platform, Layer 1 (L1), Avalan...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,binance-bridged-usdt-bnb-smart-chain,bsc-usd,Binance Bridged USDT (BNB Smart Chain),binance-bridged-usdt-bnb-smart-chain,binance-smart-chain,0,None,"[BNB Chain Ecosystem, Bridged USDT, Bridged-To...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,binancecoin,bnb,BNB,bnb,None,0,None,"[Smart Contract Platform, Exchange-based Token...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bitcoin,btc,Bitcoin,bitcoin,None,10,SHA-256,"[Smart Contract Platform, Layer 1 (L1), FTX Ho...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,bitcoin-cash,bch,Bitcoin Cash,bitcoin-cash,None,0,SHA-256,"[Smart Contract Platform, Layer 1 (L1), Bitcoi...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,bittensor,tao,Bittensor,bittensor,None,0,None,"[Artificial Intelligence (AI), Smart Contract ...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,cardano,ada,Cardano,cardano,None,0,None,"[Smart Contract Platform, Layer 1 (L1), Cardan...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,chainlink,link,Chainlink,chainlink,ethereum,0,None,"[Artificial Intelligence (AI), Business Servic...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,coinbase-wrapped-btc,cbbtc,Coinbase Wrapped BTC,coinbase-wrapped-btc,ethereum,0,None,"[Crypto-Backed Tokens, Tokenized BTC, Wrapped-...",False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
dev_rows = []
for coin in all_data:
    dev = coin.get('developer_data', {})
    dev['coin_id'] = coin.get('id')  # <-- attach parent coin ID
    dev_rows.append(dev)

# Create a DataFrame
developer_df = pd.DataFrame(dev_rows)
developer_df = developer_df.drop(
    columns=[
        "code_additions_deletions_4_weeks",
        "last_4_weeks_commit_activity_series"
    ],
    errors="ignore"
)

In [3]:
links_rows = []
for coin in all_data:
    links = coin.get('links', {})
    links['coin_id'] = coin.get('id')  # <-- attach parent coin ID
    links_rows.append(links)

# Create a DataFrame
links_df = pd.DataFrame(links_rows)



In [5]:
categories_rows = []

for coin in all_data:
    coin_id = coin.get('id')
    for cat in coin.get('categories', []):  # cat is a string
        categories_rows.append({
            'coin_id': coin_id,
            'category': cat
        })

categories_df = pd.DataFrame(categories_rows)


In [6]:
platforms_list = []
for coin in all_data:
    platforms = coin.get('platforms', {})
    if platforms:
        for platform, address in platforms.items():
            platforms_list.append({
                'coin_id': coin['id'],
                'platform': platform,
                'contract_address': address
            })

platforms_df = pd.DataFrame(platforms_list)

In [7]:
ticker_rows = []

for coin in all_data:
    coin_id = coin.get('id')
    for ticker in coin.get('tickers', []):
        # add the parent coin ID
        ticker['coin_id'] = coin_id
        ticker_rows.append(ticker)

# json_normalize to flatten nested structures
tickers_df = pd.json_normalize(
    ticker_rows,
    sep='_'
)




In [8]:
# Rename description.en → description
coin_df = coin_df.rename(columns={'description.en': 'description'})

# Select only relevant parent columns
parent_columns = [
    'id',
    'symbol',
    'name',
    'preview_listing',
    'description',
    'country_origin',
    'genesis_date',
    'last_updated'
]

coin_df = coin_df.reindex(columns=parent_columns)

In [21]:
table_name = "bronze.coin_market"
table_name_2 = "bronze.coin_basic_info"
connection_string = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={r'Akeelah\SQLEXPRESS'};"
    f"DATABASE=Crypto_DataWarehouse;"
    "Trusted_Connection=yes;"
)

connection = odbc.connect(connection_string)

cursor = connection.cursor()

params_list = [
                (
                    row['id'], row['symbol'], row['name'], row['preview_listing'], row['description'],
                    row['country_origin'], row['genesis_date'], row['last_updated']
                )
                for _, row in coin_df.iterrows()
            ]

load_query = f'''
            INSERT INTO {table_name_2} (
                id, symbol, name, preview_listing, description,
                country_origin, genesis_date, last_updated
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            '''
cursor.executemany(load_query, params_list)
connection.commit()
print(f"🆕 {len(params_list)} rows loaded for new coins succesfully..........................✅✅✅")

connection.close()
cursor.close()





🆕 50 rows loaded for new coins succesfully..........................✅✅✅


In [52]:
table_name = "bronze.coin_market"
table_name_2 = "bronze.coin_basic_info"
connection_string = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={r'Akeelah\SQLEXPRESS'};"
    f"DATABASE=Crypto_DataWarehouse;"
    "Trusted_Connection=yes;"
)

connection = odbc.connect(connection_string)

cursor = connection.cursor()


sql = (f"""
        SELECT id, Max([last_updated]) AS last_data_load
        FROM {table_name_2}
        GROUP BY id
        """
        )

cursor.execute(sql)


db_coin = cursor.fetchall()

cursor.close()

db_coin = pd.DataFrame(db_coin, columns=['coin_id','last_data_load'])


new_data_new_coin = coin_df[
    ~coin_df['id'].isin(db_coin['coin_id'])
]


old_coin_df = db_coin[
    ~db_coin['coin_id'].isin(coin_df['id'])
]

coins_to_delete = old_coin_df['coin_id'].tolist()


len(new_data_new_coin)

    
if not new_data_new_coin.empty:


    connection_string = (
        f"DRIVER={{SQL Server}};"
        f"SERVER={r'Akeelah\SQLEXPRESS'};"
        f"DATABASE=Crypto_DataWarehouse;"
        "Trusted_Connection=yes;"
    )

    connection = odbc.connect(connection_string)

    cursor = connection.cursor()

    params_list = [
                    (
                        row['id'], row['symbol'], row['name'], row['preview_listing'], row['description'],
                        row['country_origin'], row['genesis_date'], row['last_updated']
                    )
                    for _, row in new_data_new_coin.iterrows()
                ]

    load_query = f'''
                INSERT INTO {table_name_2} (
                    id, symbol, name, preview_listing, description,
                    country_origin, genesis_date, last_updated
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                '''
    cursor.executemany(load_query, params_list)
    connection.commit()
    print(f'{len(params_list)} were inserted')

else:
    print('no coin to load')

if not old_coin_df.empty:

    placeholders = ",".join("?" for _ in coins_to_delete)

    delete_sql = f"""
                    DELETE FROM {table_name_2}
                    WHERE id IN ({placeholders})
                """

    cursor.execute(delete_sql)
    deleted_rows = cursor.rowcount
    connection.commit()

    print(f"🗑️ Deleted {deleted_rows} rows for coins that are not in top 50 anymore")
else:
     print("ℹ️ No  coins to delete")






    

no coin to load
ℹ️ No  coins to delete


In [83]:


table_config = {
    "bronze.coin_basic_info": {
        "id_col": "id",
        "insert_cols": [
            "id", "symbol", "name", "preview_listing",
            "description", "country_origin",
            "genesis_date", "last_updated"
        ],
        "source_df": coin_df
    },
    "bronze.coin_categories_info": {
        "id_col": "coin_id",
        "insert_cols": ["coin_id", "category"],
        "source_df": categories_df
    },
    "bronze.coin_platform_info": {
        "id_col": "coin_id",
        "insert_cols": ["coin_id", "platform", "contract_address"],
        "source_df": platforms_df
    },
    "bronze.coin_developer_info": {
        "id_col": "coin_id",
        "insert_cols": [
            "forks", "stars", "subscribers", "total_issues",
            "pull_requests_merged", "pull_request_contributors",
            "commit_count_4_weeks", "coin_id"
        ],
        "source_df": developer_df
    },

    "bronze.coin_links_info": {
        "id_col": "coin_id",
        "insert_cols": [
            "homepage", "whitepaper", "blockchain_site", "official_forum_url",
            "chat_url", "announcement_url",
            "snapshot_url", "twitter_screen_name","bitcointalk_thread_identifier", "subreddit_url",
            "repos_url", "coin_id"

        ],
        "source_df": links_df
        
        }

}




table_name_2 = "bronze.coin_basic_info"

connection_string = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={r'Akeelah\SQLEXPRESS'};"
    f"DATABASE=Crypto_DataWarehouse;"
    "Trusted_Connection=yes;"
)

connection = odbc.connect(connection_string)
cursor = connection.cursor()

# CRITICAL: Enable fast_executemany for all inserts
cursor.fast_executemany = True



# Fetch existing coins from coin_basic_info
sql = f"""
    SELECT id, MAX([last_updated]) AS last_data_load
    FROM {table_name_2}
    GROUP BY id
"""
cursor.execute(sql)
db_coin = cursor.fetchall()
db_coin = pd.DataFrame(db_coin, columns=['coin_id', 'last_data_load'])

# Compute new and old coins based on coin_basic_info
new_data_new_coin = coin_df[~coin_df['id'].isin(db_coin['coin_id'])]
old_coin_df = db_coin[~db_coin['coin_id'].isin(coin_df['id'])]
coins_to_delete = old_coin_df['coin_id'].tolist()

for table, conf in table_config.items():
    source_df = conf['source_df']
    id_col = conf['id_col']
    insert_cols = conf['insert_cols']
    # Use db_cols if provided, otherwise use insert_cols
    db_cols = conf.get('db_cols', insert_cols)

    print(f"\n--- Processing {table} ---")

    # ---------- DELETE OLD COINS ----------
    if coins_to_delete:
        placeholders = ",".join("?" for _ in coins_to_delete)
        delete_sql = f"""
            DELETE FROM {table}
            WHERE {id_col} IN ({placeholders})
        """
        cursor.execute(delete_sql, coins_to_delete)
        deleted_rows = cursor.rowcount
        connection.commit()
        print(f'{deleted_rows} rows deleted')
    else:
        print(f'No rows to delete')

    # ---------- INSERT NEW COINS ----------
    if table == table_name_2:
        insert_df = source_df[source_df[id_col].isin(new_data_new_coin['id'])]
    else:
        if 'coin_id' in source_df.columns:
            cursor.execute(f"SELECT DISTINCT {id_col} FROM {table}")
            existing_categories = cursor.fetchall()
            existing_coin_ids = [row[0] for row in existing_categories]
            
            all_valid_coins = coin_df['id'].tolist()
            coins_needing_categories = [c for c in all_valid_coins if c not in existing_coin_ids]
            insert_df = source_df[source_df['coin_id'].isin(coins_needing_categories)]
        else:
            insert_df = source_df.copy()

    if not insert_df.empty:
        start_time = time.time()
        
        # Clean the data before insert
        insert_df_clean = insert_df[insert_cols].copy()
        
        # Replace NaN/inf with None for SQL NULL
        insert_df_clean = insert_df_clean.replace([float('inf'), float('-inf')], None)
        insert_df_clean = insert_df_clean.where(pd.notna(insert_df_clean), None)
        
      

        insert_sql = f"""
            INSERT INTO {table} ({",".join(db_cols)})
            VALUES ({placeholders})
        """
        
        try:
            cursor.executemany(insert_sql, params)
            connection.commit()
            
            elapsed = time.time() - start_time
            print(f'{len(params)} rows inserted in {elapsed:.2f} seconds')
        except Exception as e:
            print(f'Error inserting into {table}: {e}')
            print(f'First row sample: {params[0] if params else "No data"}')
            raise
    else:
        print(f'No new rows to insert')

cursor.close()
connection.close()
print("\n✓ All operations completed!")


--- Processing bronze.coin_basic_info ---
No rows to delete
No new rows to insert

--- Processing bronze.coin_categories_info ---
No rows to delete
No new rows to insert

--- Processing bronze.coin_platform_info ---
No rows to delete
No new rows to insert

--- Processing bronze.coin_developer_info ---
No rows to delete
No new rows to insert

--- Processing bronze.coin_links_info ---
No rows to delete
Error inserting into bronze.coin_links_info: ('22018', '[22018] [Microsoft][ODBC SQL Server Driver]Invalid character value for cast specification')
First row sample: ('https://app.aave.com/', 'https://github.com/aave/aave-protocol/blob/master/docs/Aave_Protocol_Whitepaper_v1_0.pdf', 'https://etherscan.io/token/0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,https://ethplorer.io/address/0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,https://polygonscan.com/token/0xd6df932a45c0f255f85145f286ea0b292b21c90b,https://snowtrace.io/token/0x63a72806098bd3d9520cc43356dd78afe5d386d9,https://nearblocks.io/tok

DataError: ('22018', '[22018] [Microsoft][ODBC SQL Server Driver]Invalid character value for cast specification')

In [89]:
# -------------------------------
# AUTO-DETECT + NORMALIZE INLINE
# -------------------------------

import pandas as pd

# 1. Detect column shapes
scalar_cols = []
list_cols = []
dict_cols = []

for col in links_df.columns:
    sample = links_df[col].dropna().head(20)

    if sample.apply(lambda x: isinstance(x, list)).any():
        list_cols.append(col)
    elif sample.apply(lambda x: isinstance(x, dict)).any():
        dict_cols.append(col)
    else:
        scalar_cols.append(col)

# -------------------------------
# 2. MAIN (SCALAR) TABLE
# -------------------------------
links_main_df = links_df[scalar_cols].copy()

# -------------------------------
# 3. LIST → CHILD TABLE
# -------------------------------
list_rows = []

for _, row in links_df.iterrows():
    coin_id = row["coin_id"]

    for col in list_cols:
        val = row[col]

        if isinstance(val, list):
            values = val
        elif isinstance(val, str):
            values = [v.strip() for v in val.split(",") if v.strip()]
        else:
            values = []

        for v in values:
            list_rows.append({
                "coin_id": coin_id,
                "attribute": col,
                "value": v
            })

links_list_df = pd.DataFrame(list_rows)

# -------------------------------
# 4. DICT → CHILD TABLE
# -------------------------------
dict_rows = []

for _, row in links_df.iterrows():
    coin_id = row["coin_id"]

    for col in dict_cols:
        val = row[col]

        if not isinstance(val, dict):
            continue

        for subtype, values in val.items():
            if not isinstance(values, list):
                values = [values]

            for v in values:
                dict_rows.append({
                    "coin_id": coin_id,
                    "attribute": col,
                    "subtype": subtype,
                    "value": v
                })

links_dict_df = pd.DataFrame(dict_rows)

links_dict_df

""


In [56]:
tickers_df

,base,target,last,volume,trust_score,bid_ask_spread_percentage,timestamp,last_traded_at,last_fetch_at,is_anomaly,...,coin_mcap_usd,market_name,market_identifier,market_has_trading_incentive,converted_last_btc,converted_last_eth,converted_last_usd,converted_volume_btc,converted_volume_eth,converted_volume_usd
0,AAVE,USDT,182.510000,2.897249e+06,green,0.015480,2025-12-15T20:08:53+00:00,2025-12-15T20:08:53+00:00,2025-12-15T20:08:53+00:00,False,...,2.781391e+09,CoinUp.io,coinup,False,0.002124,0.062235,182.49,6417.000000,188024.000000,5.513461e+08
1,AAVE,USDT,182.700000,4.406877e+05,green,0.015483,2025-12-15T20:07:48+00:00,2025-12-15T20:07:48+00:00,2025-12-15T20:08:31+00:00,False,...,2.781391e+09,Bybit,bybit_spot,False,0.002126,0.062299,182.68,977.895000,28652.000000,8.401604e+07
2,AAVE,USDT,182.890000,2.204156e+05,green,0.015470,2025-12-15T20:03:44+00:00,2025-12-15T20:03:44+00:00,2025-12-15T20:03:44+00:00,False,...,2.781391e+09,BVOX,bitvenus_spot,False,0.002126,0.062328,182.87,489.821000,14363.000000,4.214195e+07
3,AAVE,USDT,183.480000,1.758009e+05,green,0.015470,2025-12-15T20:01:17+00:00,2025-12-15T20:01:17+00:00,2025-12-15T20:01:17+00:00,False,...,2.781391e+09,Zoomex,zoomex,False,0.002130,0.062457,183.46,389.223000,11412.000000,3.351995e+07
4,AAVE,USDT,182.580000,1.145707e+05,green,0.015479,2025-12-15T20:07:11+00:00,2025-12-15T20:07:11+00:00,2025-12-15T20:08:21+00:00,False,...,2.781391e+09,Binance,binance,False,0.002125,0.062361,182.62,243.506000,7145.000000,2.092250e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,ZEC,USDT,436.818468,4.578561e-02,None,3.741612,2025-12-10T18:42:21+00:00,2025-12-10T18:42:21+00:00,2025-12-11T14:30:11+00:00,True,...,6.719545e+09,Komodo Wallet,komodo_wallet,False,0.004845,0.137019,436.84,0.000222,0.006274,2.000000e+01
3929,SYS,ZEC,0.000047,1.046999e+02,None,1.413414,2025-12-10T03:52:43+00:00,2025-12-10T03:52:43+00:00,2025-12-11T01:54:38+00:00,True,...,1.469395e+07,Komodo Wallet,komodo_wallet,False,0.004809,0.134101,436.88,0.000024,0.000665,2.170000e+00
3930,KMD,ZEC,0.000085,1.791000e+02,None,20.141551,2025-12-09T12:54:24+00:00,2025-12-09T12:54:24+00:00,2025-12-10T04:25:33+00:00,True,...,5.726935e+06,Komodo Wallet,komodo_wallet,False,0.003994,0.111545,369.38,0.000061,0.001706,5.650000e+00
3931,ZEC,INR,31738.300000,3.000000e-04,None,6.637201,2025-12-14T21:24:50+00:00,2025-12-14T21:24:50+00:00,2025-12-15T20:13:43+00:00,True,...,6.719545e+09,Giottus,giottus,False,0.004075,0.119510,349.79,0.000001,0.000036,1.049380e-01


In [57]:
tickers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933 entries, 0 to 3932
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   base                          3933 non-null   object 
 1   target                        3933 non-null   object 
 2   last                          3933 non-null   float64
 3   volume                        3933 non-null   float64
 4   trust_score                   3762 non-null   object 
 5   bid_ask_spread_percentage     3884 non-null   float64
 6   timestamp                     3933 non-null   object 
 7   last_traded_at                3933 non-null   object 
 8   last_fetch_at                 3933 non-null   object 
 9   is_anomaly                    3933 non-null   bool   
 10  is_stale                      3933 non-null   bool   
 11  trade_url                     3808 non-null   object 
 12  token_info_url                41 non-null     object 
 13  coi